In [1]:
from ultralytics import YOLO

In [3]:
model = YOLO(f"star2-yolo11n.yaml")

[128, 256]
[128, 128]
[64, 128]
[128, 256]
torch.Size([1, 128, 16, 16]) torch.Size([1, 256, 16, 16])
torch.Size([1, 128, 32, 32]) torch.Size([1, 128, 32, 32])
torch.Size([1, 64, 16, 16]) torch.Size([1, 128, 16, 16])
torch.Size([1, 128, 8, 8]) torch.Size([1, 256, 8, 8])
